In [2]:
""" 라이브러리 설치는 여기  """
!pip install happytransformer
from IPython.display import clear_output
clear_output()

In [30]:
""" 임포트는 여기에 멘션했습니다. """
import csv
import os
from datasets import load_dataset
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
from happytransformer import HappyTextToText
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer
from google.colab import files
config = T5Config.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
""" 모델 """
happy_tt = HappyTextToText("T5", "t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
""" 데이터셋 로드 """
train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[:]')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [6]:
"""데이터 검사"""
for case in train_dataset["corrections"][:2]:
    print(case)
    print(case[0])
    print("--------------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [7]:
"""데이터 전처리 작업"""
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings.
                if input_text and correction:
                    writter.writerow([input_text, correction])

In [8]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [9]:
"""데이터 미세 조정전 모델 평가"""
before_result = happy_tt.eval("eval.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [10]:
print("Before loss:", before_result.loss)

Before loss: 1.28038489818573


In [11]:
"""Training"""
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
1,1.555600,1.361047
34,0.849600,0.679001
68,0.752400,0.580555
102,0.662800,0.542612
136,0.637200,0.531399
170,0.628400,0.518125
204,0.612000,0.505470
238,0.602400,0.498149
272,0.570500,0.495834
306,0.548200,0.494440


In [12]:
"""After Training Evaluating"""
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

After loss:  0.48289430141448975


In [13]:
"""inference"""
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [14]:
""" Example1: """
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

This sentence has bad grammar and spelling!


In [15]:
""" Example2: """

example_2 = "grammar: I am enjoys, writtings articles ons AI and I also enjoyed write articling on AI."

result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI and I also enjoyed writing articles on AI.


In [31]:
#모델을 저장할 디렉토리 경로 설정
path_to_save = "/content/capstone_AI"
#디렉토리가 존재하지 않으면 생성
if not os.path.exists(path_to_save):
    os.makedirs(path_to_save)
#모델 파일 이름 설정
model_file_name = "model.pth"
#전체 파일 경로 설정
file_path = os.path.join(path_to_save, model_file_name)

In [33]:
# 다운로드할 파일 경로 설정 (capstone_AI 변수에 저장한 경로를 사용)
file_path = '/content/capstone_AI/model.pth'

In [37]:
# T5 모델과 토크나이저 로드
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
# 모델 저장
t5_model.save_pretrained(file_path)
t5_tokenizer.save_pretrained(file_path)

('/content/capstone_AI/model.pth/tokenizer_config.json',
 '/content/capstone_AI/model.pth/special_tokens_map.json',
 '/content/capstone_AI/model.pth/spiece.model',
 '/content/capstone_AI/model.pth/added_tokens.json')

In [39]:
#모델 파일 다운로드
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
#위의 모델 파일 다운로드 << 다운로드하고 나서 file.download항목 주석처리 할 것. 안하면 로컬파일에 계속 다운로드 당함.